In [77]:
import pandas as pd
import numpy as np
import os, sys
from pybaseball import batting_stats

START = 2002
END = 2022

batting_data_df = batting_stats(START, END, qual=175)
print("does this finally work after 2 days of trying to get it to work?",batting_data_df.head())

batting_data_df.to_csv("batting_stats_df.csv")



'''
from pybaseball import statcast
statcast(start_dt="2019-06-24", end_dt="2019-06-25").columns

data2 = batting_stats_range("2022-05-26", "2022-06-10")
print(data2.head())

data3 = pitching_stats_range("2022-05-26", "2022-06-10")
print(data3.head())
'''

does this finally work after 2 days of trying to get it to work?      IDfg  Season          Name Team  Age    G   AB   PA    H  1B  ...  maxEV  \
0    1109    2002   Barry Bonds  SFG   37  143  403  612  149  70  ...    NaN   
1    1109    2004   Barry Bonds  SFG   39  147  373  617  135  60  ...    NaN   
9   15640    2022   Aaron Judge  NYY   30  157  570  696  177  87  ...  118.4   
16  13611    2018  Mookie Betts  BOS   25  136  520  614  180  96  ...  110.6   
2    1109    2003   Barry Bonds  SFG   38  130  390  550  133  65  ...    NaN   

    HardHit  HardHit%  Events  CStr%   CSW%  xBA  xSLG  xwOBA  L-WAR  
0       NaN       NaN       0  0.127  0.191  NaN   NaN    NaN   12.7  
1       NaN       NaN       0  0.124  0.164  NaN   NaN    NaN   11.9  
9     246.0     0.609     404  0.169  0.287  NaN   NaN    NaN   11.2  
16    217.0     0.500     434  0.220  0.270  NaN   NaN    NaN   10.4  
2       NaN       NaN       0  0.135  0.223  NaN   NaN    NaN   10.2  

[5 rows x 320 columns

'\nfrom pybaseball import statcast\nstatcast(start_dt="2019-06-24", end_dt="2019-06-25").columns\n\ndata2 = batting_stats_range("2022-05-26", "2022-06-10")\nprint(data2.head())\n\ndata3 = pitching_stats_range("2022-05-26", "2022-06-10")\nprint(data3.head())\n'

In [78]:
batting_data_df = batting_data_df.groupby("IDfg", group_keys=False).filter(lambda x: x.shape[0] > 1)
print(batting_data_df.head())

     IDfg  Season          Name Team  Age    G   AB   PA    H  1B  ...  maxEV  \
0    1109    2002   Barry Bonds  SFG   37  143  403  612  149  70  ...    NaN   
1    1109    2004   Barry Bonds  SFG   39  147  373  617  135  60  ...    NaN   
9   15640    2022   Aaron Judge  NYY   30  157  570  696  177  87  ...  118.4   
16  13611    2018  Mookie Betts  BOS   25  136  520  614  180  96  ...  110.6   
2    1109    2003   Barry Bonds  SFG   38  130  390  550  133  65  ...    NaN   

    HardHit  HardHit%  Events  CStr%   CSW%  xBA  xSLG  xwOBA  L-WAR  
0       NaN       NaN       0  0.127  0.191  NaN   NaN    NaN   12.7  
1       NaN       NaN       0  0.124  0.164  NaN   NaN    NaN   11.9  
9     246.0     0.609     404  0.169  0.287  NaN   NaN    NaN   11.2  
16    217.0     0.500     434  0.220  0.270  NaN   NaN    NaN   10.4  
2       NaN       NaN       0  0.135  0.223  NaN   NaN    NaN   10.2  

[5 rows x 320 columns]


In [79]:
#Creating a helper function that'll be applied to dataframe that makes a new column next to WAR of current season that shows the player's WAR for the season after the current one if criteria is met

def war_next_season(player):
    player = player.sort_values("Season")
    player["next_year_WAR"] = player["WAR"].shift(-1)
    return player

#Checking to see what/if the changes were made
batting_data_df = batting_data_df.groupby("IDfg", group_keys=False).apply(war_next_season)
batting_data_df[["Name", "Season", "WAR", "next_year_WAR"]]


,Name,Season,WAR,next_year_WAR
5822,Alfredo Amezaga,2006,1.1,2.0
5217,Alfredo Amezaga,2007,2.0,1.2
5487,Alfredo Amezaga,2008,1.2,NaN
1200,Garret Anderson,2002,3.7,5.1
887,Garret Anderson,2003,5.1,0.8
...,...,...,...,...
7590,Yoshi Tsutsugo,2022,-1.3,NaN
5290,Shogo Akiyama,2020,0.4,-0.6
7486,Shogo Akiyama,2021,-0.6,NaN
7007,Ha-seong Kim,2021,0.5,3.7


In [80]:
null_values = batting_data_df.isnull().sum()
print(null_values)

completed_columns = list(batting_data_df.columns[null_values == 0])
print(completed_columns)

#Added to avoid copy warnings
batting_data_df = batting_data_df[completed_columns + ["next_year_WAR"]].copy()
print(batting_data_df)

IDfg                0
Season              0
Name                0
Team                0
Age                 0
                 ... 
xBA              7248
xSLG             7248
xwOBA            7248
L-WAR               0
next_year_WAR    1251
Length: 321, dtype: int64
['IDfg', 'Season', 'Name', 'Team', 'Age', 'G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG', 'GB', 'FB', 'LD', 'IFFB', 'Pitches', 'Balls', 'Strikes', 'IFH', 'BU', 'BUH', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS', 'ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'wOBA', 'wRAA', 'wRC', 'Bat', 'Rep', 'Pos', 'RAR', 'WAR', 'Dol', 'Spd', 'wRC+', 'WPA', '-WPA', '+WPA', 'RE24', 'REW', 'pLI', 'PH', 'WPA/LI', 'Clutch', 'FB% (Pitch)', 'FBv', 'SL%', 'SLv', 'CB%', 'CBv', 'CH%', 'CHv', 'wFB', 'wSL', 'wCB', 'wCH', 'wFB/C', 'wSL/C', 'wCB/C', 'wCH/C', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Stri

In [81]:
batting_data_df.dtypes[batting_data_df.dtypes == "object"]
del batting_data_df["Dol"]
del batting_data_df["Age Rng"]

batting_data_df["team_code"] = batting_data_df["Team"].astype("category").cat.codes
batting_data_df

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,next_year_WAR,team_code
5822,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,113,143,109,63,0,0.188,0.256,1.1,2.0,12
5217,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,112,109,113,75,0,0.175,0.227,2.0,1.2,12
5487,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,101,123,111,64,0,0.178,0.244,1.2,NaN,12
1200,2,2002,Garret Anderson,ANA,30,158,638,678,195,107,...,80,65,97,129,0,0.137,0.232,3.7,5.1,1
887,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,80,90,99,109,0,0.164,0.252,5.1,0.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7590,27459,2022,Yoshi Tsutsugo,PIT,30,50,170,193,29,23,...,125,85,118,77,124,0.181,0.267,-1.5,NaN,25
5290,27461,2020,Shogo Akiyama,CIN,32,54,155,183,38,31,...,156,107,116,73,122,0.197,0.273,0.4,-0.6,8
7486,27461,2021,Shogo Akiyama,CIN,33,88,162,183,33,25,...,147,89,118,79,125,0.197,0.284,-0.5,NaN,8
7007,27506,2021,Ha-seong Kim,SDP,25,117,267,298,54,32,...,59,137,96,88,201,0.216,0.303,0.6,3.7,26


In [82]:
#make a copy of batting data so empty cells of Next_WAR can be dropped, forecast next season
batting_full = batting_data_df.copy()
#drop next_war empty cells
batting_data_df = batting_data_df.dropna().copy()


In [83]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

#lambda
ridge_reg = Ridge(alpha=1)
split = TimeSeriesSplit(n_splits=3)

#loop thru features until 20 best valuable features are found
seq_feat_sel = SequentialFeatureSelector(ridge_reg, n_features_to_select=20, direction="forward", cv=split, n_jobs=4)

In [84]:
no_touch_columns = ["next_year_WAR", "Name", "Team", "IDfg", "Season"]
selected_columns = batting_data_df.columns[~batting_data_df.columns.isin(no_touch_columns)]

#scaling data so mean can be 1 and std =1 using min/maxing w/ sklearn
from sklearn.preprocessing import MinMaxScaler
agg_scaler = MinMaxScaler()
batting_data_df.loc[:, selected_columns] = agg_scaler.fit_transform(batting_data_df[selected_columns])

In [85]:
batting_data_df
batting_data_df.describe()

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,next_year_WAR,team_code
count,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,...,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000,5997.000000
mean,5444.225946,2011.226447,0.347087,0.645263,0.468728,0.474248,0.366172,0.290661,0.384089,0.099245,...,0.407652,0.413194,0.512143,0.476050,0.168330,0.499564,0.528299,0.327920,1.701001,0.473031
std,5213.778519,5.638641,0.143067,0.258933,0.248932,0.266452,0.185346,0.141128,0.176407,0.104256,...,0.131424,0.122710,0.131500,0.134732,0.267817,0.136423,0.116772,0.119091,1.981600,0.305958
min,1.000000,2002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.400000,0.000000
25%,1152.000000,2006.000000,0.259259,0.451613,0.252613,0.242123,0.209205,0.175926,0.241379,0.043478,...,0.320896,0.331461,0.420290,0.387755,0.000000,0.412766,0.453390,0.242236,0.200000,0.205882
50%,3692.000000,2011.000000,0.333333,0.701613,0.489547,0.495854,0.368201,0.287037,0.362069,0.086957,...,0.402985,0.404494,0.507246,0.482993,0.000000,0.497872,0.529661,0.304348,1.300000,0.470588
75%,9077.000000,2016.000000,0.444444,0.870968,0.686411,0.709784,0.514644,0.393519,0.517241,0.130435,...,0.492537,0.488764,0.594203,0.564626,0.315526,0.591489,0.605932,0.391304,2.800000,0.735294
max,27506.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.900000,1.000000


In [86]:
seq_feat_sel.fit(batting_data_df[selected_columns], batting_data_df["next_year_WAR"])



SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=4)

In [119]:
predictors = list(selected_columns[seq_feat_sel.get_support()])

In [120]:
#using cross validation won't work because timing/order of seasons matters

def back_test(data, model, predictors, start=5, step=1):
    predictions = []
    years = sorted(data["Season"].unique())

    for i in range(start, len(years), step):
        current_year = years[i]

        train_set = data[data["Season"] < current_year]
        test_set = data[data["Season"] == current_year]

        model.fit(train_set[predictors], train_set["next_year_WAR"])

        preds = model.predict(test_set[predictors])

        preds = pd.Series(preds, index=test_set.index)

        combined = pd.concat([test_set["next_year_WAR"], preds], axis=1)
        combined.columns = ["actual", "prediction"]

        predictions.append(combined)
    return pd.concat(predictions)



In [121]:
predictions = back_test(batting_data_df, ridge_reg, predictors)
predictions

from sklearn.metrics import mean_squared_error
mean_squared_error(predictions["actual"], predictions["prediction"])

2.7416107089704225

In [122]:
print(batting_data_df["next_year_WAR"].describe())

2.7416107089704225 ** .5

#sqrt of mean_squared_error is less than std which is headed in the right direction


count    5997.000000
mean        1.701001
std         1.981600
min        -3.400000
25%         0.200000
50%         1.300000
75%         2.800000
max        11.900000
Name: next_year_WAR, dtype: float64


1.65578099668115

In [123]:
def player_history(data):
    data = data.sort_values("Season")

    data["player_season"] = range(0, data.shape[0])
    #for every season a player has played will give correlation between the season number and his WAR for all prev seasons. used list to get rid of the index number
    data["war_corr"] = list(data[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"), "WAR"])
    data["war_corr"].fillna(1, inplace=True)

    #take current season War divided by next year/predicted war. the shift brings prev year war up a spot
    data["war_diff"] = data["next_year_WAR"] / data["WAR"].shift(1)
    data["war_diff"].fillna(1, inplace=True)

    data["war_diff"][data["war_diff"] == np.inf] = 1

    return data

#splitting dataframe into groups and applying player_history to columns
batting_data_df = batting_data_df.groupby("IDfg", group_keys=False).apply(player_history)

In [124]:
def group_avg(data):
    return data["WAR"] / data["WAR"].mean()


batting_data_df["war_season"] = batting_data_df.groupby("Season", group_keys=False).apply(group_avg)



In [137]:
#need to find the Nan or infinite values in one or more of these columns because as it stands I can't add these to new_predictors to run tests, it throws and " Input X contains infinity or a value too large for dtype('float6" error
#new_predictors = predictors +["player_season", "war_corr", "war_season", "war_diff"]
new_predictors = predictors 
new_p = back_test(batting_data_df, ridge_reg, new_predictors)
new_predictors

print(predictions.head())

      actual  prediction
5217     1.2    1.536638
1986     1.4    0.738450
3207    -0.1    0.637009
6079     0.6    0.937735
1139     4.8    2.237327


In [130]:

#predictions = back_test(batting_data_df, ridge_reg, new_predictors)

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [131]:

mean_squared_error(predictions["actual"], predictions["prediction"])

2.7416107089704225

In [ ]:
'''
ga = batting_data_df[batting_data_df["IDfg"] == 2 ].copy()

ga
ga["player_season"] = range(0, ga.shape[0])
list(ga[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"), "WAR"])
'''
#testing the playerhistory function step by step 

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Events,CStr%,CSW%,L-WAR,next_year_WAR,team_code,player_season,war_corr,war_diff,war_season
1200,2,2002,Garret Anderson,ANA,0.407407,0.959677,0.864111,0.834163,0.719665,0.453704,...,0.0,0.365957,0.402542,0.440994,5.1,0.029412,0,1.000000,1.000000,1.332308
887,2,2003,Garret Anderson,ANA,0.444444,0.967742,0.864111,0.825871,0.744770,0.509259,...,0.0,0.480851,0.487288,0.527950,0.8,0.029412,1,1.000000,1.814085,1.615520
2652,2,2004,Garret Anderson,ANA,0.481481,0.588710,0.522648,0.497512,0.460251,0.412037,...,0.0,0.531915,0.563559,0.260870,-0.2,0.029412,2,-0.661143,-0.378824,0.793657
4344,2,2005,Garret Anderson,LAA,0.518519,0.830645,0.754355,0.709784,0.585774,0.472222,...,0.0,0.421277,0.512712,0.198758,0.1,0.441176,3,-0.836562,0.383333,0.599679
4108,2,2006,Garret Anderson,LAA,0.555556,0.822581,0.698606,0.684909,0.539749,0.444444,...,0.0,0.442553,0.491525,0.217391,1.4,0.441176,4,-0.836312,7.043750,0.662220
1986,2,2007,Garret Anderson,LAA,0.592593,0.556452,0.479094,0.456053,0.422594,0.310185,...,0.0,0.442553,0.461864,0.298137,1.4,0.441176,5,-0.692192,6.440000,0.914482
3460,2,2008,Garret Anderson,LAA,0.629630,0.854839,0.722997,0.693201,0.585774,0.504630,...,0.0,0.527660,0.512712,0.298137,-1.1,0.441176,6,-0.595013,-3.689583,0.915753
